In [1]:
#pip install deap

Note: If deap not installed, run above line.

In [2]:
import numpy, random
from deap import algorithms, base, creator, tools, gp
import matplotlib.pyplot as plt

In [3]:
x =numpy.linspace(-2, 2.75, 20)
x

array([-2.  , -1.75, -1.5 , -1.25, -1.  , -0.75, -0.5 , -0.25,  0.  ,
        0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,  2.25,
        2.5 ,  2.75])

In [4]:
yVals = numpy.array([37.00000,24.16016, 15.06250, 8.91016,5.00000,
                     2.72266,1.56250,1.09766,1.00000,1.03516,
                     1.06250,1.03516,1.00000,1.09766,1.56250,
                     2.72266,5.00000,8.91016,15.06250,24.16016])
yVals

array([37.     , 24.16016, 15.0625 ,  8.91016,  5.     ,  2.72266,
        1.5625 ,  1.09766,  1.     ,  1.03516,  1.0625 ,  1.03516,
        1.     ,  1.09766,  1.5625 ,  2.72266,  5.     ,  8.91016,
       15.0625 , 24.16016])

In [6]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Tree", gp.PrimitiveTree, fitness=creator.FitnessMin)

pset = gp.PrimitiveSet(name="MAIN", arity=1)
pset.addPrimitive(numpy.add, arity=2)
pset.addPrimitive(numpy.subtract, arity=2)
pset.addPrimitive(numpy.multiply, arity=2)
pset.addPrimitive(numpy.negative, arity=1)

pset.addEphemeralConstant("one", lambda: random.randint(-1, 1))

def evaluateRegression(individual, points, pset):
    func = gp.compile(expr=individual, pset=pset)
    sqerrors = (func(points)-yVals)**2
    return (numpy.sqrt(numpy.sum(sqerrors) / len(points)),)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Tree, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluateRegression, points=numpy.linspace(-2, 2.75, 20), pset=pset)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.register("select", tools.selTournament, tournsize=3)

if __name__ == "__main__":
    pop = toolbox.population(n=300)
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, verbose=False)
    best = tools.selBest(pop, k=1)[0]
    print(best)
    func = gp.compile(expr=best, pset=pset)
    plt.plot(x, yVals, label = "actual")
    plt.plot(x, func(x), label = "regression function")
    plt.legend()
    
    fitness = evaluateRegression(best, x, pset)
    print('Fitness', fitness)

C:\Users\Kevin\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
C:\Users\Kevin\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Tree' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Exception: Ephemerals with different functions should be named differently, even between psets.